# Pell方程式

In [1]:
import sympy
sympy.init_printing(use_latex=True)

sympy.var('D x0 x1 y0 y1')
x = x0 + 2*x1
y = y0 + 2*y1
f = (x**2 - D*y**2 -1)**2
sympy.expand(f)
#sympy.latex(sympy.expand(f))

# グレブナー基底を確認しておく

In [2]:
sympy.groebner([x0**2-x0, x1**2-x1, y0**2-y0, y1**2-y1], x0, x1, y0, y1)

# グレブナー基底を使った次数の簡約

In [3]:
f = sympy.rem(f, x0**2-x0, x0)
f = sympy.rem(f, x1**2-x1, x1)
f = sympy.rem(f, y0**2-y0, y0)
f = sympy.rem(f, y1**2-y1, y1)
sympy.expand(f)
#sympy.latex(sympy.expand(f))

# 3変数以上の項を簡約

まずは4次式でのHを求める

In [4]:
sympy.var('L a b c')
Hc = L*(a*b -2*(a+b)*c +3*c)
sympy.var('z0 z1')
H = z1*y1 + Hc.subs([(a, x0), (b, x1), (c, z0)]) + Hc.subs([(a, z0), (b, y0), (c, z1)])
sympy.expand(H)
#sympy.latex(sympy.expand(H))

3次式を求める

In [5]:
x0x1y0 = H.subs([(y1, 1)])
sympy.expand(x0x1y0)
#sympy.latex(sympy.expand(x0x1y0))

In [6]:
x0x1y1 = H.subs([(y0, 1)])
sympy.expand(x0x1y1)
#sympy.latex(sympy.expand(x0x1y1))

In [7]:
x0y0y1 = H.subs([(x1, 1)])
sympy.expand(x0y0y1)
#sympy.latex(sympy.expand(x0y0y1))

In [8]:
x1y0y1 = H.subs([(x0, 1)])
sympy.expand(x1y0y1)
#sympy.latex(sympy.expand(x1y0y1))

4次の項を消去

In [9]:
f = sympy.poly(f, x0, x1, y0, y1)
f3deg = f + (f.coeff_monomial(x0*x1*y0*y1)*H - f.coeff_monomial(x0*x1*y0*y1)*x0*x1*y0*y1)
sympy.expand(f3deg)
#sympy.latex(sympy.expand(f3deg))

3次の項を消去

In [10]:
f2deg = f3deg + (8*D*x0*x1*y0 - 8*D*x0x1y0) + (32*D*x0*x1*y1 - 32*D*x0x1y1) + (8*D*x0*y0*y1 - 8*D*x0y0y1) + (32*D*x1*y0*y1 - 32*D*x1y0y1)
sympy.expand(f2deg)
#sympy.latex(sympy.expand(f2deg))

Dに具体的な値を代入し、QUBOを導出

In [11]:
temp = sympy.poly((sympy.poly(f2deg, D).eval(D, 3)))
qubo = sympy.poly((sympy.poly(temp, L).eval(L, 1)))
sympy.expand(qubo)
#sympy.latex(sympy.expand(qubo))

# 量子アニーリングでQUBOの解を求める

In [12]:
from dimod import *
from dwave_qbsolv import QBSolv
var_list = [x0, x1, y0, y1, z0, z1]

linear = dict()
for var1 in var_list:
    linear[var1] = qubo.coeff_monomial(var1)

quadratic = dict()
for index, var1 in enumerate(var_list):
    for var2 in var_list[index + 1:]:
        quadratic[(var1, var2)] = qubo.coeff_monomial(var1*var2)

bqm = BinaryQuadraticModel(linear, quadratic, qubo.coeff_monomial(1), dimod.BINARY)
result = QBSolv().sample(bqm)
for s, e in result.data(['sample','energy']):
    print(s,'\t E = ', e)

{x0: 1, x1: 1, y0: 0, y1: 1, z0: 0, z1: 1} 	 E =  -1376.0
